---
title: "The Great Electric Vehicle Innovation Race: A Patent Analysis of US, China, and EU Competition (2014-2024)"
author: "Qing Ye"
date: "2025-10-06"
abstract: |
  This chapter examines the competitive landscape of electric vehicle (EV) innovation through a comprehensive patent analysis spanning 2014-2024. By analyzing patent filings across seven core EV technology domains—Battery Technology, EV Propulsion & Charging, Autonomous Driving, Hybrid & Energy Management, Safety & ADAS, Thermal Management, and Infotainment & Connectivity—we uncover distinct strategic approaches among the United States, China, and the European Union. Our findings reveal a shifting power dynamic: while the EU commanded 42.2% of patent share in 2014, this declined to 33.5% by 2023 and 28.4% in 2024 (partial year). Meanwhile, China surged from 8.7% to 25.0%, demonstrating a dual strategy of battery dominance (42.1% share in 2024) and digital experience focus (24.2% in infotainment). The US maintained leadership at 44-49% share, particularly in autonomous driving (55%) and infotainment (60%). We argue that these patterns reflect fundamentally different visions: China treating EVs as consumer electronics with rapid iteration cycles, the US pursuing technology-driven disruption, and the EU leveraging traditional automotive engineering excellence. The chapter concludes with strategic recommendations for EU competitiveness, emphasizing the need to close software gaps while capitalizing on strengths in thermal management, safety systems, and sustainable manufacturing. All patent share calculations represent proportions among these three regions only, excluding other major automotive nations.
keywords: ["electric vehicles", "patent analysis", "innovation competition", "China EV strategy", "EU automotive policy", "battery technology", "autonomous driving"]
---

In [ ]:
# Setup: Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings("ignore")

# Configure Plotly to output PNG images for static rendering
# This ensures compatibility with PDF output
pio.renderers.default = "plotly_mimetype+png"
pio.kaleido.scope.default_format = "png"
pio.kaleido.scope.default_width = 1000
pio.kaleido.scope.default_height = 500

# Introduction

The global electric vehicle (EV) revolution represents one of the most consequential industrial transformations of the 21st century. As nations race to decarbonize transportation and reshape automotive supply chains, innovation—measured through patent activity—serves as a critical proxy for competitive positioning and future market leadership.

This chapter analyzes patent filing patterns across the United States, China, and the European Union from 2014 to 2024, focusing on seven core EV technology domains. Unlike previous studies that examine patent office locations, our methodology counts patents by **assignee/inventor country**, providing a more accurate picture of which region's companies are truly innovating. This distinction is crucial: Chinese companies file 93.7% of their patents domestically, while US and EU companies file 45-60% internationally.

## Research Questions

This analysis addresses three central questions:

1. **How has patent share distribution evolved among the US, China, and EU from 2014-2024?**
2. **What distinct strategic approaches do these regions demonstrate across different EV technology domains?**
3. **What policy and business strategies should the EU pursue to maintain competitiveness in the EV innovation race?**

## Methodology Overview

Our analysis draws from Google's Public Patent Dataset (BigQuery), focusing on patents filed by companies and inventors from the US, China, and EU member states. We categorize patents into seven technology domains using Cooperative Patent Classification (CPC) codes:

1. **Battery Technology** - Energy storage systems (H01M, H01G11)
2. **EV Propulsion & Charging** - Motors, motor control, charging infrastructure (B60L, H02K, H02P, H02J7, H02M)
3. **Autonomous Driving** - Self-driving and vehicle control systems (B60W, G05D1)
4. **Hybrid & Energy Management** - Hybrid powertrains, energy optimization (B60K6, B60W20, F02D)
5. **Safety & ADAS** - Advanced driver assistance, safety systems (B60R, B60Q, G08G)
6. **Thermal Management** - Battery cooling, heat management (B60H, F28D)
7. **Infotainment & Connectivity** - Vehicle displays, dashboards, connectivity, V2X (B60K35, B60K37, H04W4, G07C5, H04N7/18)

**Important note**: All patent share percentages represent proportions **among these three regions only**—not global patent share. Other major automotive nations (Japan, South Korea, etc.) are excluded from this analysis to maintain focus on the trilateral competition.

The time period spans 2014-2024, with 2024 data clearly marked as incomplete (partial year data).

## Data Sources and Processing

We queried the `patents-public-data.patents.publications` table in Google BigQuery, joining with CPC classification definitions to categorize patents. The EU region aggregates all 27 current member states. Technical implementation details are provided in Appendix A.

In [ ]:
# Load and prepare data
file_path = "../data/bquxjob_1552e56a_199b9009593.csv"
data = pd.read_csv(file_path)

# Pivot data for overall country comparison
pivot_data = data.pivot_table(
    index="year",
    columns="country",
    values="patent_count",
    aggfunc="sum",
).fillna(0)

total_patents = pivot_data.sum(axis=1)
pivot_data_percentage = pivot_data.div(pivot_data.sum(axis=1), axis=0) * 100

In [ ]:
# Define plotting functions for visualizations
def plot_plotly_stacked_bar_and_line(
    pivot_data_pct, pivot_data_abs, absolute_data, title, filename=None
):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    for country in pivot_data_pct.columns:
        fig.add_trace(
            go.Bar(
                x=pivot_data_pct.index,
                y=pivot_data_pct[country],
                name=country,
                text=pivot_data_pct[country].round(1).astype(str) + "%",
                textposition="inside",
                customdata=pivot_data_abs[country],
                hovertemplate="<b>%{fullData.name}</b><br>"
                + "Year: %{x}<br>"
                + "Patents: %{customdata:,}<br>"
                + "Share: %{y:.1f}%<br>"
                + "<extra></extra>",
            ),
            secondary_y=False,
        )
    fig.add_trace(
        go.Scatter(
            x=absolute_data.index,
            y=absolute_data,
            mode="lines+markers",
            name="Total Patents",
            line=dict(color="red", dash="dot", width=2),
            opacity=0.5,
            hovertemplate="<b>Total Patents</b><br>"
            + "Year: %{x}<br>"
            + "Patents: %{y:,}<br>"
            + "<extra></extra>",
        ),
        secondary_y=True,
    )
    fig.update_yaxes(title_text="Percentage (%)", secondary_y=False)
    fig.update_yaxes(title_text="Total Patents", secondary_y=True)
    fig.update_layout(
        title=title,
        barmode="stack",
        xaxis_title="Year",
        legend_title="Region",
        hovermode="x unified",
        height=500,
    )
    fig.show()
    if filename:
        fig.write_html(filename)


def plot_per_application_area(data, filename=None):
    application_areas = sorted(data["application_area"].unique())
    for app_area in application_areas:
        subset = data[data["application_area"] == app_area]
        pivot_data = subset.pivot_table(
            index="year",
            columns="country",
            values="patent_count",
            aggfunc="sum",
        ).fillna(0)
        total_patents = pivot_data.sum(axis=1)
        pivot_data_percentage = pivot_data.div(pivot_data.sum(axis=1), axis=0) * 100
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        for country in pivot_data_percentage.columns:
            fig.add_trace(
                go.Bar(
                    x=pivot_data_percentage.index,
                    y=pivot_data_percentage[country],
                    name=country,
                    text=pivot_data_percentage[country].round(1).astype(str) + "%",
                    textposition="inside",
                    customdata=pivot_data[country],
                    hovertemplate="<b>%{fullData.name}</b><br>"
                    + "Year: %{x}<br>"
                    + "Patents: %{customdata:,}<br>"
                    + "Share: %{y:.1f}%<br>"
                    + "<extra></extra>",
                ),
                secondary_y=False,
            )
        fig.add_trace(
            go.Scatter(
                x=total_patents.index,
                y=total_patents,
                mode="lines+markers",
                name="Total Patents",
                line=dict(color="red", dash="dot", width=2),
                opacity=0.5,
                hovertemplate="<b>Total Patents</b><br>"
                + "Year: %{x}<br>"
                + "Patents: %{y:,}<br>"
                + "<extra></extra>",
            ),
            secondary_y=True,
        )
        fig.update_yaxes(title_text="Percentage (%)", secondary_y=False)
        fig.update_yaxes(title_text="Total Patents", secondary_y=True)
        fig.update_layout(
            title=f"Patent Share in {app_area}: US vs China vs EU (2014-2024)",
            barmode="stack",
            xaxis_title="Year",
            legend_title="Region",
            hovermode="x unified",
            height=450,
        )
        fig.show()
        if filename:
            fig.write_html(
                f"{app_area.replace(' ', '_').replace('&', 'and')}_patent_share.html"
            )

# Empirical Findings: The Shifting Landscape of EV Innovation

## Overall Patent Share Trends (2014-2024)

Figure 1 presents the overall patent share distribution across the three regions over the decade-long study period. The visualization reveals three distinct trajectories that reshape our understanding of the global EV innovation landscape.

In [ ]:
# Figure 1: Overall Patent Share Evolution
plot_plotly_stacked_bar_and_line(
    pivot_data_percentage,
    pivot_data,
    total_patents,
    "Figure 1: Core EV Patent Share by Region (2014-2024)",
)

### Key Observations

**The EU's Decline**: European patent share fell from 42.2% (2014) to 33.5% (2023), continuing downward to 28.4% in 2024 (partial year data). This represents a loss of 14 percentage points over the decade—a concerning trajectory for a region historically dominant in automotive innovation.

**China's Meteoric Rise**: Chinese patent share grew from 8.7% (2014) to 22.5% (2023) and 25.0% (2024*), an increase of 16 percentage points. This growth reflects massive government investment, domestic market incentives, and strategic focus on EV as a leapfrog technology.

**US Sustained Leadership**: The United States maintained relatively stable leadership, ranging from 44.0% (2023) to 46.6% (2024*). This consistency masks important shifts across technology domains, as detailed in subsequent sections.

**Total Volume Trends**: The red dotted line shows total patent volume peaked around 2017-2018, then declined—suggesting market consolidation and maturation. The 2024 decline may reflect incomplete data or a genuine slowdown.

## Technology Domain Analysis

Figure 2 breaks down patent share across the seven core EV technology domains, revealing divergent regional strategies and competitive positioning.

In [ ]:
# Figure 2: Patent Share by Technology Domain
plot_per_application_area(data)

### Domain-Specific Insights

#### 1. Autonomous Driving: US Dominance (55%)

The United States commands 55% patent share in autonomous driving technologies (2023), driven by Silicon Valley leaders like Waymo, Tesla, and Cruise. The EU holds only 31%, reflecting a strategic focus on Advanced Driver Assistance Systems (ADAS) rather than full autonomy. China's share fluctuates around 14%, constrained by AI capability gaps despite heavy investment.

**Implication**: Autonomous driving represents a critical weakness for the EU—a technology domain increasingly central to vehicle differentiation and consumer value.

#### 2. Battery Technology: China's Strategic Victory (42% in 2024*)

China's battery patent share surged from 7.3% (2014) to 42.1% (2024*), overtaking both the US and EU. This dominance extends beyond patents to manufacturing capacity (CATL, BYD, Gotion) and supply chain control. The EU's share declined from 46% (2014) to 25.3% (2024*), while the US stabilized around 33%.

**Implication**: Battery technology is the "semiconductor of the EV era"—China's leadership here provides structural advantages across the entire value chain.

#### 3. Thermal Management & Safety: EU Strongholds (44-50%)

The EU maintains leadership in thermal management (~50% in 2023, 44% in 2024*) and safety/ADAS systems (~44% in 2023, 47% in 2024*). These strengths reflect traditional automotive engineering excellence from companies like Bosch, Continental, and OEM research divisions.

**Implication**: These represent the EU's competitive moat—areas where decades of automotive expertise provide defensible advantages.

#### 4. Infotainment & Connectivity: The Digital Experience Gap

The US dominates infotainment with ~60% patent share, driven by Tesla's software-defined vehicle approach and tech industry crossover. China holds ~24%, investing heavily in digital cockpits and connectivity. The EU lags at 19% (2023), declining to 16% (2024*).

**Implication**: Consumer-facing digital experiences increasingly drive purchase decisions—the EU's weakness here threatens brand competitiveness even where mechanical engineering excels.

#### 5. Hybrid & Energy Management: EU's Transitional Strength (50%)

The EU commands ~50% patent share in hybrid systems and energy management, reflecting expertise in transitional powertrains from traditional OEMs (BMW, Mercedes, Volkswagen). However, this may represent a "burning platform"—strength in a declining technology category.

**Implication**: The EU must leverage hybrid expertise to fund pure EV R&D, not rely on it as a long-term competitive position.

#### 6. EV Propulsion & Charging: Balanced Competition

Patent share in propulsion and charging technologies shows relative balance: US (38-48%), EU (30-38%), China (20-25%). This suggests a commoditizing technology domain where fundamental innovations have plateaued.

**Implication**: Differentiation will come from system integration and efficiency optimization rather than breakthrough propulsion innovations.

## Summary of Empirical Patterns

Three key patterns emerge from the data:

1. **Divergent Regional Strategies**: China pursues vertical integration (batteries + digital), the US focuses on software/autonomy, and the EU leverages traditional engineering (thermal, safety, hybrids).

2. **The Software Divide**: Digital technologies (autonomy, infotainment) increasingly favor the US and China over the EU, threatening long-term competitiveness as EVs become "software-defined."

3. **China's Dual Strategy**: Unlike conventional automotive competition, China simultaneously dominates hardware (batteries) and invests heavily in software (infotainment)—a smartphone-industry playbook.

The next section examines the strategic implications of these patterns, focusing on policy recommendations for EU competitiveness.

# EU Competitive Landscape Analysis

The patent analysis reveals Europe's current positioning in the global EV race—maintaining competitive strengths in traditional engineering domains while facing critical gaps in emerging software-defined vehicle technologies.

**Current Position (2023) - Share Among Three Regions:**
- **US**: 44.0% (leading)
- **EU**: 33.5% (declining from 42.2% in 2014)
- **China**: 22.5% (surging from 8.7% in 2014)

## EU Innovation Strengths

### Thermal Management & Safety Systems (45-50% patent share)
The EU maintains leadership in thermal management (~50% in 2023, 44% in 2024*) and safety/ADAS systems (~44% in 2023, 47% in 2024*). These strengths reflect decades of automotive engineering excellence from companies like Bosch, Continental, and traditional OEMs. Thermal management expertise is particularly critical for battery longevity and fast-charging capabilities.

### Hybrid & Energy Management (~50% patent share)
European automakers dominate hybrid technologies through BMW, Mercedes, and Volkswagen innovation. However, this represents strength in a potentially declining category as markets shift to pure EVs—a "burning platform" requiring strategic transition.

## Critical Weaknesses

### Software & Digital Experience Gap
- **Autonomous Driving**: Only 31% share (2023) vs US 55% dominance
- **Infotainment & Connectivity**: Declining from 19% (2023) to 16% (2024*)—severely lagging behind US (60%) and China (24%)
- **Implication**: Consumer-facing digital experiences increasingly drive purchase decisions, threatening EU brand competitiveness despite mechanical engineering excellence

### Battery Technology Decline
While maintaining meaningful presence (30% in 2023, 25% in 2024*) through Northvolt and partnerships, the EU faces China's accelerating dominance (38% → 42%). The challenge is achieving manufacturing scale and supply chain control.

### Overall Patent Trajectory
EU patent share declined from 42% (2014) → 34% (2023) → 28%* (2024 partial), indicating slower R&D investment pace compared to competitors.

## The Competitive Context

**US Strategy**: Software and autonomy leadership, leveraging Silicon Valley tech industry crossover

**China's Dual Approach**: 
- Battery dominance (42% in 2024*) for supply chain control
- Digital experience focus (24% infotainment) following smartphone industry playbook
- Rapid model cycles and aggressive pricing

The following case study examines China's consumer electronics approach in detail, revealing a fundamentally different competitive paradigm that challenges traditional automotive business models.

# Case Study: China's "EVs as Consumer Electronics" Strategy

## The Smartphone-on-Wheels Paradigm Shift

Our patent data reveals a critical strategic divergence: **China is treating EVs as consumer electronics, not traditional automobiles**. This explains both their patent patterns and competitive positioning.

### China's Dual Strategy: Battery Dominance + Digital Experience

#### Battery Technology: Steady Growth to Market Leadership

China's battery patent share trajectory (among three regions):
- **2014**: 7.3% (minor player)
- **2018**: 17.4% (emerging)
- **2020**: 30.0% (parity)
- **2023**: 37.9% (leading)
- **2024***: 42.1% (dominant)

**Pattern**: Consistent, aggressive growth—treating batteries as core infrastructure (like semiconductors in phones)

#### Infotainment & Connectivity: Strategic Focus

China's infotainment patent share (among three regions):
- **2014**: 15.0% (baseline)
- **2017**: 18.1% (accelerating)
- **2020**: 24.1% (major push)
- **2022**: 25.4% (peak)
- **2023-2024***: ~24% (stabilizing at high level)

**Key Insight**: China grew infotainment share from 15% → 24% while US held steady at ~60% and EU declined from 23% → 16%

### The Consumer Electronics Business Model

Evidence from industry trends:

**Rapid Model Cycles** (like smartphone releases):
- BYD: 19 new models (2017-2023)
- NIO: 9 new models
- XPeng: 6 new models
- Tesla: 5 models (for comparison)

**Consumer Electronics Features**:
- **Multiple screens**: 2-3 displays standard (like tablets)
- **AR glasses**: NIO's $350 AR headset integration
- **Smartphone integration**: XPeng's phone-to-car ecosystem
- **Karaoke systems**: Entertainment-first mindset
- **OTA updates**: Continuous software improvement (like iOS updates)

**Cost Reduction Focus**:
- XPeng SEPA 2.0: 70% cost reduction in ADAS, 85% in infotainment
- Focus on volume and affordability over premium pricing

### Strategic Implications: Different Race, Different Rules

**Traditional Auto Model (EU/US)**:
- Long development cycles (5-7 years)
- Emphasis on mechanical excellence
- Premium pricing for quality
- Safety and durability first
- Slower software innovation

**Consumer Electronics Model (China)**:
- Rapid iteration (1-2 year cycles)
- Software/digital experience focus
- Aggressive pricing
- Feature-rich for consumer appeal
- Quality/profitability trade-offs

### What 2024* Data Reveals (Incomplete Year)

Despite incomplete data, 2024 shows (share among three regions):

**Overall Patent Distribution**:
- US: 46.6% (12,605 patents)
- EU: 28.4% (7,675 patents)
- CN: 25.0% (6,777 patents)

**Infotainment Specific**:
- US: 59.5% (3,616 patents)
- CN: 24.2% (1,472 patents)
- EU: 16.3% (989 patents)

**Battery Technology**:
- **CN: 42.1% (2,567 patents)** - Highest share among three regions
- US: 32.6% (1,989 patents)
- EU: 25.3% (1,541 patents)

### Critical Observation

China maintains high infotainment focus (~24%) **while simultaneously dominating batteries (42%)**. This dual strategy is unique:
- **Hardware foundation**: Battery dominance ensures supply chain control
- **Software differentiation**: Infotainment focus drives consumer preference
- **Ecosystem play**: Like Apple's hardware + software integration

### Market Reality Check: The Profitability Challenge

Despite patent activity, Chinese EV startups face financial challenges:

**Financial Struggles**:
- NIO: $3 billion net loss (2023)
- NIO, XPeng, Great Wall: Missed sales targets 2 years running
- Response: Launching cheaper mass-market brands (2024)

**Why the disconnect?**
- Patents ≠ Profitability
- Consumer electronics model requires massive scale
- Price competition eroding margins
- Market consolidation likely

### EU Response Strategy: Don't Play China's Game

**What EU Should NOT Do:**
- Race China on rapid model cycles (plays to their strength)
- Compete on lowest-cost infotainment (loses quality advantage)
- Treat EVs as disposable consumer goods (contradicts European values)

**What EU SHOULD Do:**

1. **Reframe the Competition**
   - Position EVs as "Premium Sustainable Mobility" not gadgets
   - Emphasize durability, repairability, lifecycle value
   - European quality vs. Chinese quantity

2. **Selective Digital Investment**
   - Do invest in safety-critical software (autonomous driving, ADAS)
   - Do invest in privacy-first connectivity (European values)
   - Don't copy multi-screen excess
   - Focus on user experience quality over feature quantity

3. **Leverage Sustainability Advantage**
   - Circular economy expertise (battery recycling)
   - Thermal management leadership = longer battery life = less waste
   - Appeal to environmentally conscious consumers globally

4. **Premium Positioning**
   - Combine thermal management + battery tech = superior systems
   - Market "10-year lifespan" vs. "3-year upgrade cycle"
   - Target premium segment (like European watches vs. smartwatches)

### Two Different Futures

The patent data reveals **two fundamentally different visions** for EVs:

**Chinese Vision**: Fast-moving consumer goods
- Rapid refresh cycles
- Digital experience paramount
- Volume and affordability
- Ecosystem lock-in (apps, services, connectivity)

**European Opportunity**: Sustainable premium mobility
- Long-life products
- Engineering excellence
- Quality and safety
- Environmental responsibility

The EU doesn't need to become China. It needs to offer a **compelling alternative** that appeals to consumers who value:
- Products that last beyond 3 years
- Privacy and data security
- Environmental sustainability
- Engineering quality over feature bloat

**The Critical Question**: Can the EU move fast enough to execute this differentiated strategy before market share erosion becomes irreversible?

# Strategic Recommendations for EU Competitiveness



Building on the empirical findings and China case study insights, this section presents actionable strategies for EU policymakers and industry leaders to revitalize European competitiveness in the global EV race.



## The EU's Strategic Imperative



The data reveals three concurrent challenges:

1. **Declining patent share**: 42% (2014) → 28%* (2024)

2. **Software gap**: Lagging in autonomous driving (31%) and infotainment (16%)

3. **Strategic fragmentation**: 27 different national approaches undermining collective competitiveness



Yet the EU retains crucial advantages: world-class engineering heritage, leadership in thermal management (44%) and safety (47%), strong industrial base, and a unified market of 450M people.



## Priority 1: Close the Software & Digital Gap



**Problem**: EU losing the software-defined vehicle race



**Recommended Actions**:



1. **Create an EU Digital Mobility Alliance**

   - Pool resources from BMW, Mercedes, Stellantis, VW for common software platform

   - Partner with European tech champions (SAP, Siemens, Bosch Software)

   - Counter Tesla's vertical integration advantage through collective action



2. **Accelerate Autonomous Driving R&D**

   - Establish pan-European testing corridors

   - Increase funding for AI and sensor fusion research

   - Partner with universities and startups (Wayve, Oxbotica)

   - Target 40% patent share in autonomous driving by 2030



3. **Develop Compelling Digital Experiences**

   - Build EU infotainment platforms competitive with Tesla/Rivian

   - Focus on privacy-first, user-centric design (European values as differentiator)

   - Integrate with EU green mobility ecosystems



## Priority 2: Leverage Existing Strengths



**Thermal Management & Safety Leadership** (45-50% patent share)



**Recommended Actions**:



1. **Battery System Integration**

   - Combine thermal management expertise with battery technology

   - Create superior battery systems (longer life, faster charging, safer)

   - Market "European Quality" as premium positioning globally



2. **Safety as Global Standard**

   - Maintain regulatory leadership in vehicle safety

   - Innovate in crashworthiness for heavy EV batteries

   - Export EU safety standards to emerging markets



3. **Hybrid Technology Bridge**

   - Leverage 50% hybrid patent share to fund pure EV R&D

   - Don't abandon transitional technology prematurely

   - Use hybrid revenue to finance software investments



## Priority 3: Secure Battery Supply Chain



**Current Position**: 30% (2023) → 25%* (2024), competing with China's 38% → 42%*



**Recommended Actions**:



1. **Scale European Battery Manufacturing**

   - Accelerate support for Northvolt, Freyr, ACC (Stellantis/Mercedes JV)

   - Attract Asian battery makers to manufacture in EU with regulatory incentives

   - Target 40% global battery patent share by 2030



2. **Next-Generation Chemistry Leadership**

   - Invest in solid-state batteries (QuantumScape European partnerships)

   - Sodium-ion for affordable EVs

   - Silicon anode technology for range improvements



3. **Circular Economy Advantage**

   - Lead global innovation in battery recycling technology

   - Create closed-loop battery supply chains

   - Leverage regulatory advantage in sustainability as market differentiator



## Priority 4: Coordinated EU Industrial Policy



**Problem**: Fragmentation undermining collective competitiveness



**Recommended Actions**:



1. **EU-Wide EV Moonshot Program**

   - €100B over 10 years (matching US IRA, China subsidies)

   - Focus on software, batteries, charging infrastructure

   - Coordinate national programs (Germany, France, Italy) under unified framework



2. **Regulatory Harmonization**

   - Single EU market for EV testing and approval

   - Unified charging standards and grid integration

   - Streamlined permitting for gigafactories (reduce from years to months)



3. **Protect & Nurture Innovation Ecosystem**

   - European DARPA for mobility innovation

   - Easier access to capital for EV/battery startups

   - Prevent brain drain to US/China through competitive funding



## Differentiated Positioning: The European Alternative



**Don't compete on China's consumer electronics playbook**. Instead, position European EVs as **"Premium Sustainable Mobility"**:



- **Durability**: 10-year lifespan vs 3-year upgrade cycles

- **Privacy**: Data security and European values

- **Safety**: Uncompromising standards

- **Environmental responsibility**: Circular economy leadership

- **Engineering quality**: Precision over feature bloat



This positioning appeals to consumers who value longevity, privacy, sustainability, and quality—a substantial global market segment underserved by the Chinese volume approach.



## Implementation Roadmap: 2025-2030



### Phase 1: Stabilize (2025-2026)

- Stop patent share erosion

- Launch EU Digital Mobility Alliance

- Defend leadership in thermal management & safety



### Phase 2: Compete (2027-2029)

- Reach 40% patent share among three regions

- Competitive autonomous driving platforms in production

- European battery gigafactories at scale (500 GWh capacity)



### Phase 3: Lead (2030+)

- Define next-generation EV standards globally

- Export EU technology and regulatory frameworks

- Establish sustainable competitive advantage



## Summary



The data shows **the EU has not lost the EV race**, but stands at a critical decision point. The next five years will determine whether European automotive leadership—built over more than a century—continues into the electric era.



**The Path Forward**: EU can reclaim leadership by:

1. Playing to strengths (thermal, safety, quality engineering)

2. Fixing critical gaps (software, autonomy, digital experience)

3. Coordinated action (pan-European programs, not 27 separate strategies)

4. Speed (move faster than traditional automotive timelines)



The window remains open, but is closing rapidly.

# Conclusion and Policy Implications

This chapter's patent analysis of US, China, and EU electric vehicle innovation from 2014-2024 reveals a dynamic, rapidly shifting competitive landscape with profound implications for industrial policy, corporate strategy, and the future of automotive manufacturing.

## Key Findings Summary

1. **Regional Divergence**: The three regions pursue fundamentally different innovation strategies:
   - **US**: Software and autonomy leadership (55% autonomous driving, 60% infotainment)
   - **China**: Dual strategy of battery dominance (42%) and digital experience (24% infotainment)
   - **EU**: Traditional engineering excellence (50% thermal management, 47% safety) but software gaps

2. **EU's Concerning Trajectory**: European patent share declined from 42.2% (2014) to 28.4% (2024*), with particular weakness in software-defined vehicle technologies crucial for future competitiveness.

3. **China's Strategic Success**: Patent share grew from 5.9% to 25.0%, demonstrating successful execution of a smartphone-industry playbook applied to automotive: control batteries (hardware), invest in infotainment (software), pursue rapid iteration.

4. **The Software Imperative**: Across all three regions, digital technologies (autonomous driving, infotainment, connectivity) increasingly differentiate competitive positioning—areas where the EU significantly lags.

## Theoretical Contributions

This analysis contributes to innovation studies and industrial policy literature in three ways:

**First**, we demonstrate the importance of **measuring innovation by inventor/assignee country** rather than patent office location. Chinese companies file 93.7% domestically while US/EU companies file 45-60% internationally—traditional patent metrics thus systematically misrepresent competitive positioning.

**Second**, we identify **distinct innovation strategies** reflecting different industrial paradigms: China's consumer electronics approach (rapid iteration, volume, affordability) versus the EU's automotive engineering paradigm (quality, safety, longevity). These aren't just different speeds—they represent fundamentally different theories of value creation.

**Third**, we highlight the **strategic coordination gap** as a critical EU weakness. While individual European companies maintain technical excellence, fragmented national policies and lack of pan-European platforms (especially in software) undermine collective competitiveness.

## Policy Recommendations

### For the European Union:

1. **Urgent Software Investment**: Create a €20-30B EU Digital Mobility Alliance pooling resources from major OEMs, tech companies, and research institutions. Focus on autonomous driving, infotainment platforms, and V2X connectivity.

2. **Battery Supply Chain Security**: Accelerate support for European battery manufacturing (Northvolt, ACC, Freyr) while investing in next-generation chemistry (solid-state, sodium-ion). Target 40% global patent share by 2030.

3. **Regulatory Harmonization**: Eliminate fragmentation through single EU market for EV testing, approval, and charging infrastructure. Fast-track permitting for gigafactories.

4. **Differentiated Positioning**: Don't compete on China's consumer electronics playbook. Instead, position European EVs as "Premium Sustainable Mobility"—emphasizing durability, privacy, safety, and environmental responsibility.

5. **Coordinated Industrial Policy**: Launch €100B EU EV Moonshot Program (2025-2035) matching US IRA and Chinese subsidies, with coordinated national programs rather than 27 separate strategies.

### For the United States:

1. **Maintain Software Leadership**: Continue investment in autonomous driving and AI while ensuring talent pipeline through immigration and STEM education.

2. **Address Battery Gap**: While US companies lead in battery patents (33%), manufacturing capacity lags China significantly. The IRA's battery manufacturing incentives must accelerate.

3. **Strengthen Supply Chains**: Reduce dependence on Chinese battery materials and components through domestic mining, processing, and manufacturing.

### For China:

1. **Path to Profitability**: Patent leadership doesn't guarantee business success—Chinese EV startups face consolidation pressure. Focus must shift from pure innovation to sustainable business models.

2. **Quality over Quantity**: As China moves upmarket, the rapid-iteration consumer electronics model may need refinement to address safety, durability, and premium positioning.

## Limitations and Future Research

This analysis has several limitations suggesting future research directions:

**First**, patent counts measure innovation inputs, not commercialization success. Future research should link patent patterns to market outcomes, manufacturing capacity, and firm profitability.

**Second**, we exclude other major automotive nations (Japan, South Korea) to focus on US-China-EU competition. Comprehensive global analysis would provide additional context.

**Third**, 2024 data is incomplete. Future updates will reveal whether observed trends (China's battery acceleration, EU's infotainment decline) continue.

**Fourth**, patent quality varies—counting treats all patents equally. Citation analysis and patent value metrics could refine these findings.

**Fifth**, technology domains overlap (autonomous driving requires batteries, infotainment, sensors). Network analysis could reveal integration capabilities across domains.

## Final Reflection

The electric vehicle revolution represents more than automotive electrification—it embodies a fundamental reshaping of value creation, competitive advantage, and industrial leadership in the 21st century economy.

**The EU faces a moment of truth**: Either adapt rapidly to software-defined mobility while leveraging traditional strengths, or risk becoming a premium manufacturer in a market increasingly defined by Chinese volume and American software.

**China's challenge** is converting patent and manufacturing leadership into sustainable profitability and global brand equity.

**The US must** maintain software advantages while addressing hardware (battery) gaps that could undermine competitiveness.

The window for strategic repositioning remains open, but is closing. The next five years will determine whether European automotive leadership—built over more than a century—continues into the electric era, or becomes a historical artifact of the internal combustion age.

The data is clear: **the race is not over, but the EU must run faster and smarter to stay in it.**

---

**Important Notes**:
1. **Patent Share Calculation**: All percentages represent patent share **among these three regions only** (US, China, EU)—not global patent share. Other countries (Japan, South Korea, etc.) are excluded from this analysis.
2. **Patent vs. Market Share**: This analysis focuses on patent activity as a proxy for innovation leadership. Patent share does not directly equal market share—Chinese companies may have lower patent counts but higher vehicle sales due to domestic market advantages. However, long-term competitiveness requires both innovation (patents) and commercialization (market share).
3. **2024 Data**: Marked with asterisk (*) throughout to indicate incomplete year—percentages will shift as full-year data becomes available.

---

# Appendix A: Data Sources and Methodology

## Data Source: Google Public Patent Dataset

This analysis draws from Google's Public Patent Dataset hosted on BigQuery, specifically the `patents-public-data.patents.publications` table. This dataset provides comprehensive global patent information including:

- Patent publication numbers
- Filing dates
- Assignee information (harmonized company/inventor data)
- Cooperative Patent Classification (CPC) codes
- Bibliographic metadata

## BigQuery SQL Query

**IMPORTANT**: This query counts patents by **assignee/inventor country** rather than patent office location, providing accurate attribution of innovation to regions.

### Key Methodological Decisions:

1. **Assignee Country vs. Patent Office**: We use `assignee_harmonized.country_code` instead of `p.country_code` to count who filed patents, not where they were filed. This addresses the bias where Chinese companies file 93.7% domestically while US/EU companies file 45-60% internationally.

2. **EU Aggregation**: All 27 current EU member states are mapped to single "EU" region for comparative analysis.

3. **Time Period**: 2014-2024, with 2024 data incomplete as of analysis date.

4. **Technology Domains**: Seven categories using CPC classification codes (see below).

### Complete SQL Query:

```sql
WITH cpc_mapping AS (
  SELECT
    symbol,
    titleFull,
    CASE
      -- Battery Technology (Energy Storage)
      WHEN symbol LIKE 'H01M%' THEN 'Battery Technology'
      WHEN symbol LIKE 'H01G11%' THEN 'Battery Technology'
      
      -- Electric Vehicle Propulsion & Charging
      WHEN symbol LIKE 'B60L%' THEN 'EV Propulsion & Charging'
      WHEN symbol LIKE 'H02K%' THEN 'EV Propulsion & Charging'
      WHEN symbol LIKE 'H02P%' THEN 'EV Propulsion & Charging'
      WHEN symbol LIKE 'H02J7%' THEN 'EV Propulsion & Charging'
      WHEN symbol LIKE 'H02M%' THEN 'EV Propulsion & Charging'
      
      -- Autonomous Driving & Vehicle Control
      WHEN symbol LIKE 'B60W%' THEN 'Autonomous Driving'
      WHEN symbol LIKE 'G05D1%' THEN 'Autonomous Driving'
      
      -- Hybrid Systems & Energy Management
      WHEN symbol LIKE 'B60K6%' THEN 'Hybrid & Energy Management'
      WHEN symbol LIKE 'B60W20%' THEN 'Hybrid & Energy Management'
      WHEN symbol LIKE 'F02D%' THEN 'Hybrid & Energy Management'
      
      -- Vehicle Safety & ADAS
      WHEN symbol LIKE 'B60R%' THEN 'Safety & ADAS'
      WHEN symbol LIKE 'B60Q%' THEN 'Safety & ADAS'
      WHEN symbol LIKE 'G08G%' THEN 'Safety & ADAS'
      
      -- Thermal Management (Battery Cooling)
      WHEN symbol LIKE 'B60H%' THEN 'Thermal Management'
      WHEN symbol LIKE 'F28D%' THEN 'Thermal Management'
      
      -- Infotainment & Connectivity
      WHEN symbol LIKE 'B60K35%' THEN 'Infotainment & Connectivity'
      WHEN symbol LIKE 'B60K37%' THEN 'Infotainment & Connectivity'
      WHEN symbol LIKE 'H04W4%' THEN 'Infotainment & Connectivity'
      WHEN symbol LIKE 'G07C5%' THEN 'Infotainment & Connectivity'
      WHEN symbol LIKE 'H04N7/18%' THEN 'Infotainment & Connectivity'
      
      ELSE NULL
    END AS application_area
  FROM `patents-public-data.cpc.definition`
  WHERE symbol IS NOT NULL
),

-- EU country codes for mapping
eu_countries AS (
  SELECT country_code FROM UNNEST([
    'AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR',
    'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL',
    'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE'
  ]) AS country_code
),

patent_data AS (
  SELECT
    p.publication_number,
    p.filing_date,
    cpc_code.code AS cpc_code,
    assignee.country_code AS assignee_country
  FROM `patents-public-data.patents.publications` p,
    UNNEST(p.cpc) AS cpc_code,
    UNNEST(p.assignee_harmonized) AS assignee
  WHERE p.filing_date IS NOT NULL
    AND p.filing_date >= 20140101
    AND p.filing_date <= 20241231
    AND assignee.country_code IS NOT NULL
),

-- Aggregate to region level (CN, US, EU)
patent_data_regional AS (
  SELECT DISTINCT
    pd.publication_number,
    pd.filing_date,
    pd.cpc_code,
    CASE
      WHEN pd.assignee_country = 'CN' THEN 'CN'
      WHEN pd.assignee_country = 'US' THEN 'US'
      WHEN pd.assignee_country IN (SELECT country_code FROM eu_countries) THEN 'EU'
      ELSE NULL
    END AS region
  FROM patent_data pd
  WHERE pd.assignee_country IN ('CN', 'US')
     OR pd.assignee_country IN (SELECT country_code FROM eu_countries)
)

SELECT
  pdr.region AS country,
  cm.application_area,
  CAST(pdr.filing_date / 10000 AS INT64) AS year,
  COUNT(DISTINCT pdr.publication_number) AS patent_count
FROM patent_data_regional pdr
JOIN cpc_mapping cm ON pdr.cpc_code = cm.symbol
WHERE cm.application_area IS NOT NULL
  AND pdr.region IS NOT NULL
GROUP BY country, application_area, year
ORDER BY country, application_area, year
```

## CPC Classification Mapping

**Core EV Technology Domains (7 Categories):**

1. **Battery Technology** - Energy storage systems
   - H01M: Batteries and accumulators
   - H01G11: Hybrid capacitors (supercapacitors)

2. **EV Propulsion & Charging** - Motors, motor control, charging infrastructure
   - B60L: Electric propulsion of vehicles
   - H02K: Rotating electric machines
   - H02P: Control of electric motors
   - H02J7: Charging or depolarizing batteries
   - H02M: Power conversion (inverters, converters)

3. **Autonomous Driving** - Self-driving and vehicle control systems
   - B60W: Conjoint control of vehicle sub-units
   - G05D1: Control of position, course, or altitude of land vehicles

4. **Hybrid & Energy Management** - Hybrid powertrains, energy optimization
   - B60K6: Hybrid vehicle arrangements
   - B60W20: Control systems for hybrid vehicles
   - F02D: Controlling combustion engines (for hybrid integration)

5. **Safety & ADAS** - Advanced driver assistance, safety systems
   - B60R: Vehicles safety arrangements
   - B60Q: Lighting devices for vehicles
   - G08G: Traffic control systems

6. **Thermal Management** - Battery cooling, heat management
   - B60H: Heating, cooling, ventilation in vehicles
   - F28D: Heat exchangers

7. **Infotainment & Connectivity** - Vehicle displays, dashboards, connectivity, V2X
   - B60K35: Instruments for vehicles
   - B60K37: Dashboards
   - H04W4: Services for wireless communication networks
   - G07C5: Registering or indicating vehicle operation
   - H04N7/18: Closed circuit television systems (backup cameras, etc.)

## Data Processing Notes

- **Filing date format**: Stored as integer YYYYMMDD, divided by 10000 to extract year
- **EU member states**: All 27 current EU countries included (as of 2024)
- **Multiple assignees**: Patents with co-inventors from different regions counted for each region (realistic for collaborative R&D)
- **Global filing capture**: Counts patents filed anywhere in world by companies from these regions, not just domestic filings
- **No "Others" category**: Patents not matching specific categories excluded entirely to maintain focus on core EV technologies

## Data Limitations

1. **2024 incomplete**: Partial year data as of publication date
2. **Patent lag**: 18-month publication delay means recent filings may not yet appear
3. **Regional exclusions**: Japan, South Korea, and other automotive nations excluded to focus on trilateral competition
4. **Patent quality**: All patents weighted equally; citation analysis could refine
5. **Technology overlaps**: Some patents may fit multiple categories; CPC codes assigned hierarchically

## Reproducibility

This analysis is fully reproducible using Google BigQuery's public patent dataset. The complete SQL query, data processing code, and visualization functions are provided in this notebook. Data was extracted on [insert date] and reflects patent publications available as of that date.

---

# References

## China EV Industry and Strategy

Bloomberg. (2024, March 10). China EV Makers Woo Buyers With In-Car Beds, Kitchens, and Karaoke. *Bloomberg News*. https://www.bloomberg.com/news/articles/2024-03-10/5-unusual-electric-car-features-in-china-from-gaming-system-to-full-sized-beds

Digital Creative. (2024). Top 10 User-Centric HMI Designs: How China Is Leading The Way. https://digitalcreative.cn/blog/latest-automotive-hmi-design-trends

Mixed News. (2022). Augmented Reality: Nreal & Nio show new AR headset. https://mixed-news.com/en/augmented-reality-nreal-nio-show-new-ar-headset/

NIO Inc. (2024, March 5). NIO Inc. Reports Unaudited Fourth Quarter and Full Year 2023 Financial Results. *Investor Relations News Release*. https://ir.nio.com/news-releases/news-release-details/nio-inc-reports-unaudited-fourth-quarter-and-full-year-2023/

Research in China. (2024). China Passenger Car Cockpit Multi/Dual Display Research Report, 2023-2024. http://www.researchinchina.com/Htmls/Report/2024/74971.html

TechCrunch. (2023, April 16). XPeng unveils new EV platform designed to cut production costs. https://techcrunch.com/2023/04/16/xpeng-unveils-new-ev-platform-designed-to-cut-production-costs/

Wikipedia. (2024). List of BYD Auto vehicles. https://en.wikipedia.org/wiki/List_of_BYD_Auto_vehicles

Wikipedia. (2024). Nio Inc. https://en.wikipedia.org/wiki/Nio_Inc.

Wikipedia. (2024). XPeng. https://en.wikipedia.org/wiki/XPeng

XPeng Inc. (2023, April 16). XPENG Presents Next-Gen Technology Architecture – SEPA2.0. *Investor Relations News Release*. https://ir.xiaopeng.com/news-releases/news-release-details/xpeng-presents-next-gen-technology-architecture-sepa20

## Patent Analysis and Innovation Studies

Google. (2024). Public Patent Dataset. *BigQuery*. patents-public-data.patents.publications. https://console.cloud.google.com/marketplace/product/google_patents_public_datasets/google-patents-public-data

## EV Market and Policy (To be completed)

*Additional references for IEA reports, WIPO data, EU automotive policy documents, US IRA legislation, and academic literature on innovation economics and industrial policy will be added in final manuscript.*

---